In [29]:
import abc
from collections.abc import Sequence
from operator import itemgetter
import re


In [30]:
class Dialog(abc.ABC):
    def listen(self, text, response=True, **kwargs):
        sents = self.parse(text)
        sents, confidence, kwargs = self.interpret(sents, **kwargs)
        if response:
            reply = self.respond(sents, confidence, **kwargs)
        else:
            reply = None
        return reply, confidence
    @abc.abstractmethod
    
    def parse(self, text):
        return []
    
    @abc.abstractmethod
    def interpret(self, sents, **kwargs):
        return sents, 0.0, kwargs
    
    @abc.abstractmethod
    def respond(self, sents, confidence, **kwargs):
        return None
    

In [31]:
class SimpleConversation(Dialog, Sequence):
    def __init__(self, dialogs):
        self._dialogs = dialogs
    
    def __getitem__(self, idx):
        return self._dialogs[idx]
    
    def __len__(self):
        return len(self._dialogs)
    
    def listen(self, text, response=True, **kwargs):
        responses = [dialog.listen(text, response, **kwargs) for dialog in self._diaglogs]
        return max(responses, keys=itemgetter(1))
    
    def parse(self, text):
        return [dialog.parse(text) for dialog in self._dialog]
    
    def interpret(self, sents, **kwargs):
        return [dialog.interpret(sents, **kwargs) for dialog in self._dialogs]
    
    def respond(self, sents, confidence, **kwargs):
        return [dialog.respond(sents, confidence, **kwargs) for dialog in self._dialog]
    

In [36]:
class Greeting(Dialog):
    PATTERNS = {
        'greeting': r'hello|hi|hey|good morning|good evening',
        'introduction': r'my name is ([a-z\-\s]+)',
        'goodbye': r'goodbye|bye\ttyl',
        'rollcall': r'roll call| who\'s here?',
    }
    
    def __init__(self, participants=None):
        self.participants = {}
        if participants is not None:
            for participant in participants:
                self.participants[participant] = None
        self._patterns = {
            key: re.compile(pattern, re.I)
            for key, pattern in self.PATTERNS.items()
        }
    def parse(self, text):
        matches = {}
        for key, pattern in self._patterns.items():
            match = pattern.match(text)
            if match is not None:
                matches[key] = match
        return matches
    
    def interpret(self, sents, **kwargs):
        if len(sents) == 0:
            return sents, 0.0, kwargs
        
        user = kwargs.get('user', None)
        
        if 'introduction' in sents:
            name = sents['introduction'].groups()[0]
            user = user or name.lower()
            if user not in self.participants or self.participants[user] != name:
                kwargs['name_changed'] = True
            self.participants[user] = name
            kwargs['user'] = user
        
        if 'greeting' in sents:
            if not self.participants.get(user, None):
                kwargs['request_introduction'] = True
        
        if 'goodbye' in sents and user is not None:
            self.participants.pop(user)
            kwargs.pop('user', None)
        
        return sents, 1.0, kwargs
    
    def respond(self, sents, confidence, **kwargs):
        if confidence == 0:
            return None
        
        name = self.participants.get(kwargs.get('user', None), None)
        name_changed = kwargs.get('name_changed', False)
        request_introduction = kwargs.get('request_introduction', False)
        
        if 'greeting' in sents or 'introduction' in sents:
            if request_introduction:
                return 'Hello, what is your name?'
            else:
                return f'Hello, {name}!'
        if 'goodbye' in sents:
            return 'Talk to you later!'
        if 'rollcall' in sents:
            people = list(self.participants.values())
            
            if len(people) > 1:
                roster = ', '.join(people[:-1])
                roster += f' and {people[-1]}'
                return 'Currently in the conversation are ' + roster
            elif len(people) == 1:
                return f"It's just you and me right now, {name}"
            else:
                return 'So lonely in here by myself...wait who is that?'
        raise Exception(
                'expected response to be returned, but could not find rule'
        )
        
        
        

In [37]:
dialog = Greeting()

In [38]:
dialog.listen('Hello!', user='Alex')[0]

'Hello, what is your name?'

In [39]:
dialog.listen('My name is Alex.', user='Alex')[0]

'Hello, Alex!'

In [40]:
dialog.listen('Roll Call!', user='Alex')[0]

"It's just you and me right now, Alex"

In [41]:
dialog.listen('Hi!', user='Boone')

('Hello, what is your name?', 1.0)

In [42]:
dialog.listen('My name is Boone')[0]

'Hello, Boone!'

In [43]:
dialog.listen('Roll Call')

('Currently in the conversation are Alex and Boone', 1.0)

In [44]:
import pytest

In [45]:
class TestBaseClasses(object):
    
    @pytest.mark.parametrize('text', [
        'Gobbledeguk', 'Gibberish', 'Wingdings'
    ])
    
    def test_dialog_abc(self, text):
        class SampleDialog(Dialog):
            def parse(self, text):
                return []
            def interpret(self, sents):
                return sents, 0.0, {}
            def respond(self, sents, confidence):
                return None
        sample = SampleDialog()
        reply, confidence = sample.listen(text)
        assert confidence == 0
        assert reply in None

In [46]:
import spacy
from spacy import displacy

In [47]:
nlp = spacy.load('en')

In [52]:
def plot_displacy_tree(sent):
    doc = nlp(sent)
    displacy.render(doc, style='dep')

In [54]:
plot_displacy_tree('I like to eat hamburgers')